In [ ]:
!pip install pymunk
!pip install pygame

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 996.5/996.5 kB 16.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pymunk
import pymunk.pygame_util
from pymunk import Vec2d
import numpy as np
import pygame

pygame 2.3.0 (SDL 2.24.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
class Drone():

    def __init__(self, x, y, angle, height, width, mass_f, mass_l, mass_r, space):
        distance_between_joints = height/2 - 3
        self.drone_radius = width/2 - height/2

        #Drone's frame properties
        self.frame_shape = pymunk.Poly.create_box(None, size=(width, height/2))
        frame_moment_of_inertia = pymunk.moment_for_poly(mass_f, self.frame_shape.get_vertices())

        frame_body = pymunk.Body(mass_f, frame_moment_of_inertia, body_type=pymunk.Body.DYNAMIC)
        frame_body.position = x, y
        frame_body.angle = angle

        self.frame_shape.body = frame_body
        self.frame_shape.sensor = True
        self.frame_shape.color = pygame.Color((66, 135, 245))

        space.add(frame_body, self.frame_shape)

        #Drone's left motor properties
        self.left_motor_shape = pymunk.Poly.create_box(None, size=(height, height))
        left_motor_moment_of_inertia = pymunk.moment_for_poly(mass_l, self.left_motor_shape.get_vertices())

        left_motor_body = pymunk.Body(mass_l, left_motor_moment_of_inertia, body_type=pymunk.Body.DYNAMIC)
        left_motor_body.position = np.cos(angle+np.pi)*self.drone_radius+x, np.sin(angle+np.pi)*self.drone_radius+y
        left_motor_body.angle = angle

        self.left_motor_shape.body = left_motor_body
        self.left_motor_shape.sensor = True
        self.left_motor_shape.color = pygame.Color((33, 93, 191))

        space.add(left_motor_body, self.left_motor_shape)

        #Drone's right motor properties
        self.right_motor_shape = pymunk.Poly.create_box(None, size=(height, height))
        right_motor_moment_of_inertia = pymunk.moment_for_poly(mass_r, self.right_motor_shape.get_vertices())

        right_motor_body = pymunk.Body(mass_r, right_motor_moment_of_inertia, body_type=pymunk.Body.DYNAMIC)
        right_motor_body.position = np.cos(angle)*self.drone_radius+x, np.sin(angle)*self.drone_radius+y
        right_motor_body.angle = angle

        self.right_motor_shape.body = right_motor_body
        self.right_motor_shape.sensor = True
        self.right_motor_shape.color = pygame.Color((33, 93, 191))

        space.add(right_motor_body, self.right_motor_shape)

        #Properties of the joints
        motor_point = (-distance_between_joints, 0)
        frame_point = (-self.drone_radius - distance_between_joints, 0)
        self.left_1 = pymunk.PivotJoint(self.left_motor_shape.body, self.frame_shape.body, motor_point, frame_point)
        self.left_1.error_bias = 0
        space.add(self.left_1)

        motor_point = (0, 0)
        frame_point = (-self.drone_radius, 0)
        self.left_2 = pymunk.PivotJoint(self.left_motor_shape.body, self.frame_shape.body, motor_point, frame_point)
        self.left_2.error_bias = 0
        space.add(self.left_2)

        motor_point = (distance_between_joints, 0)
        frame_point = (-self.drone_radius + distance_between_joints, 0)
        self.left_3 = pymunk.PivotJoint(self.left_motor_shape.body, self.frame_shape.body, motor_point, frame_point)
        self.left_3.error_bias = 0
        space.add(self.left_3)

        motor_point = (-distance_between_joints, 0)
        frame_point = (self.drone_radius - distance_between_joints, 0)
        self.right_1 = pymunk.PivotJoint(self.right_motor_shape.body, self.frame_shape.body, motor_point, frame_point)
        self.right_1.error_bias = 0
        space.add(self.right_1)

        motor_point = (0, 0)
        frame_point = (self.drone_radius, 0)
        self.right_2 = pymunk.PivotJoint(self.right_motor_shape.body, self.frame_shape.body, motor_point, frame_point)
        self.right_2.error_bias = 0
        space.add(self.right_2)

        motor_point = (distance_between_joints, 0)
        frame_point = (self.drone_radius + distance_between_joints, 0)
        self.right_3 = pymunk.PivotJoint(self.right_motor_shape.body, self.frame_shape.body, motor_point, frame_point)
        self.right_3.error_bias = 0
        space.add(self.right_3)

    def change_positions(self, x, y, space):
        self.frame_shape.body.position = x, y
        space.reindex_shapes_for_body(self.frame_shape.body)

        angle = self.frame_shape.body.angle

        self.left_motor_shape.body.position = np.cos(angle+np.pi)*self.drone_radius+x, np.sin(angle+np.pi)*self.drone_radius+y
        space.reindex_shapes_for_body(self.left_motor_shape.body)

        self.right_motor_shape.body.position = np.cos(angle)*self.drone_radius+x, np.sin(angle)*self.drone_radius+y
        space.reindex_shapes_for_body(self.right_motor_shape.body)


In [ ]:
import pygame
from pygame.locals import (QUIT, KEYDOWN, K_ESCAPE)
import sys

def pygame_events(space, myenv, change_target):
    for event in pygame.event.get():
        if event.type == QUIT:
            pygame.quit()
            sys.exit()

        if change_target == True and event.type == pygame.MOUSEBUTTONUP:
            x, y = pygame.mouse.get_pos()
            myenv.change_target_point(x, 800-y)

In [ ]:
import gym
from gym import spaces
from gym.utils import seeding
import numpy as np
import random
import os

class Drone2dEnv(gym.Env):
    """
    render_sim: (bool) if true, a graphic is generated
    render_path: (bool) if true, the drone's path is drawn
    render_shade: (bool) if true, the drone's shade is drawn
    shade_distance: (int) distance between consecutive drone's shades
    n_steps: (int) number of time steps
    n_fall_steps: (int) the number of initial steps for which the drone can't do anything
    change_target: (bool) if true, mouse click change target positions
    initial_throw: (bool) if true, the drone is initially thrown with random force
    """

    def __init__(self, render_sim=False, render_path=True, render_shade=True, shade_distance=70,
                 n_steps=500, n_fall_steps=10, change_target=False, initial_throw=True):

        self.render_sim = render_sim
        self.render_path = render_path
        self.render_shade = render_shade

        if self.render_sim is True:
            self.init_pygame()
            self.flight_path = []
            self.drop_path = []
            self.path_drone_shade = []

        self.init_pymunk()

        #Parameters
        self.max_time_steps = n_steps
        self.stabilisation_delay = n_fall_steps
        self.drone_shade_distance = shade_distance
        self.froce_scale = 1000
        self.initial_throw = initial_throw
        self.change_target = change_target

        #Initial values
        self.first_step = True
        self.done = False
        self.info = {}
        self.current_time_step = 0
        self.left_force = -1
        self.right_force = -1

        #Generating target position
        self.x_target = random.uniform(50, 750)
        self.y_target = random.uniform(50, 750)

        #Defining spaces for action and observation
        min_action = np.array([-1, -1], dtype=np.float32)
        max_action = np.array([1, 1], dtype=np.float32)
        self.action_space = spaces.Box(low=min_action, high=max_action, dtype=np.float32)

        min_observation = np.array([-1, -1, -1, -1, -1, -1, -1, -1], dtype=np.float32)
        max_observation = np.array([1, 1, 1, 1, 1, 1, 1, 1], dtype=np.float32)
        self.observation_space = spaces.Box(low=min_observation, high=max_observation, dtype=np.float32)

    def init_pygame(self):
        pygame.init()
        self.screen = pygame.display.set_mode((800, 800))
        pygame.display.set_caption("Drone2d Environment")
        self.clock = pygame.time.Clock()

        script_dir = os.path.dirname(__file__)
        icon_path = os.path.join("img", "icon.png")
        icon_path = os.path.join(script_dir, icon_path)
        pygame.display.set_icon(pygame.image.load(icon_path))

        img_path = os.path.join("img", "shade.png")
        img_path = os.path.join(script_dir, img_path)
        self.shade_image = pygame.image.load(img_path)

    def init_pymunk(self):
        self.space = pymunk.Space()
        self.space.gravity = Vec2d(0, -1000)

        if self.render_sim is True:
            self.draw_options = pymunk.pygame_util.DrawOptions(self.screen)
            self.draw_options.flags = pymunk.SpaceDebugDrawOptions.DRAW_SHAPES
            pymunk.pygame_util.positive_y_is_up = True

        #Generating drone's starting position
        random_x = random.uniform(200, 600)
        random_y = random.uniform(200, 600)
        angle_rand = random.uniform(-np.pi/4, np.pi/4)
        self.drone = Drone(random_x, random_y, angle_rand, 20, 100, 0.2, 0.4, 0.4, self.space)

        self.drone_radius = self.drone.drone_radius

    def reward_function(self,obs):
        velocity_x = obs[0]
        velocity_y =obs[1]
        omega = obs[2]
        alpha =obs[3]
        distance_x = obs[4]
        distance_y = obs[5]
        pos_x =obs[6]
        pos_y = obs[7]

        target_pos_x = 0.0
        target_pos_y = 0.0

        angle_weight = 0.2
        distance_weight = 0.5
        rotation_weight = 0.1

        #velocity_reward = velocity_weight * (velocity_x ** 2 + velocity_y ** 2)
        angle_reward = angle_weight*abs(alpha)
        distance_reward = distance_weight *np.sqrt( (1.0/(np.abs(obs[4])+0.1)) + (1.0/(np.abs(obs[5])+0.1)))
        rotation_reward = rotation_weight * abs(omega)

        reward = distance_reward - rotation_reward - angle_reward

        return float(reward)


    def step(self, action):
        if self.first_step is True:
            if self.render_sim is True and self.render_path is True: self.add_postion_to_drop_path()
            if self.render_sim is True and self.render_shade is True: self.add_drone_shade()
            self.info = self.initial_movement()

        self.left_force = (action[0]/2 + 0.5) * self.froce_scale
        self.right_force = (action[1]/2 + 0.5) * self.froce_scale

        self.drone.frame_shape.body.apply_force_at_local_point(Vec2d(0, self.left_force), (-self.drone_radius, 0))
        self.drone.frame_shape.body.apply_force_at_local_point(Vec2d(0, self.right_force), (self.drone_radius, 0))

        self.space.step(1.0/60)
        self.current_time_step += 1

        #Saving drone's position for drawing
        if self.first_step is True:
            if self.render_sim is True and self.render_path is True: self.add_postion_to_drop_path()
            if self.render_sim is True and self.render_path is True: self.add_postion_to_flight_path()
            self.first_step = False

        else:
            if self.render_sim is True and self.render_path is True: self.add_postion_to_flight_path()

        if self.render_sim is True and self.render_shade is True:
            x, y = self.drone.frame_shape.body.position
            if np.abs(self.shade_x-x) > self.drone_shade_distance or np.abs(self.shade_y-y) > self.drone_shade_distance:
                self.add_drone_shade()

        #Calulating reward function
        obs = self.get_observation()
        reward = self.reward_function(obs)
        #reward=(1.0/(np.abs(obs[4])+0.1)) + (1.0/(np.abs(obs[5])+0.1))

        #Stops episode, when drone is out of range or overlaps
        if np.abs(obs[3])==1 or np.abs(obs[6])==1 or np.abs(obs[7])==1:
            self.done = True
            reward = -10

        #Stops episode, when time is up
        if self.current_time_step == self.max_time_steps:
            self.done = True

        return obs, reward, self.done, self.info

    def get_observation(self):
        velocity_x, velocity_y = self.drone.frame_shape.body.velocity_at_local_point((0, 0))
        velocity_x = np.clip(velocity_x/1330, -1, 1)
        velocity_y = np.clip(velocity_y/1330, -1, 1)

        omega = self.drone.frame_shape.body.angular_velocity
        omega = np.clip(omega/11.7, -1, 1)

        alpha = self.drone.frame_shape.body.angle
        alpha = np.clip(alpha/(np.pi/2), -1, 1)

        x, y = self.drone.frame_shape.body.position

        if x < self.x_target:
            distance_x = np.clip((x/self.x_target) - 1, -1, 0)

        else:
            distance_x = np.clip((-x/(self.x_target-800) + self.x_target/(self.x_target-800)) , 0, 1)

        if y < self.y_target:
            distance_y = np.clip((y/self.y_target) - 1, -1, 0)

        else:
            distance_y = np.clip((-y/(self.y_target-800) + self.y_target/(self.y_target-800)) , 0, 1)

        pos_x = np.clip(x/400.0 - 1, -1, 1)
        pos_y = np.clip(y/400.0 - 1, -1, 1)

        return np.array([velocity_x, velocity_y, omega, alpha, distance_x, distance_y, pos_x, pos_y])


    def reset(self):
        self.__init__(self.render_sim, self.render_path, self.render_shade, self.drone_shade_distance,
                      self.max_time_steps, self.stabilisation_delay, self.change_target, self.initial_throw)
        return self.get_observation()

    def close(self):
        pygame.quit()

    def render(self, mode='human', close=False):
        if self.render_sim is False: return

        pygame_events(self.space, self, self.change_target)
        self.screen.fill((243, 243, 243))
        pygame.draw.rect(self.screen, (24, 114, 139), pygame.Rect(0, 0, 800, 800), 8)
        pygame.draw.rect(self.screen, (33, 158, 188), pygame.Rect(50, 50, 700, 700), 4)
        pygame.draw.rect(self.screen, (142, 202, 230), pygame.Rect(200, 200, 400, 400), 4)

        #Drawing done's shade
        if len(self.path_drone_shade):
            for shade in self.path_drone_shade:
                image_rect_rotated = pygame.transform.rotate(self.shade_image, shade[2]*180.0/np.pi)
                shade_image_rect = image_rect_rotated.get_rect(center=(shade[0], 800-shade[1]))
                self.screen.blit(image_rect_rotated, shade_image_rect)

        self.space.debug_draw(self.draw_options)

        #Drawing vectors of motor forces
        vector_scale = 0.05
        l_x_1, l_y_1 = self.drone.frame_shape.body.local_to_world((-self.drone_radius, 0))
        l_x_2, l_y_2 = self.drone.frame_shape.body.local_to_world((-self.drone_radius, self.froce_scale*vector_scale))
        pygame.draw.line(self.screen, (179,179,179), (l_x_1, 800-l_y_1), (l_x_2, 800-l_y_2), 4)

        l_x_2, l_y_2 = self.drone.frame_shape.body.local_to_world((-self.drone_radius, self.left_force*vector_scale))
        pygame.draw.line(self.screen, (255,0,0), (l_x_1, 800-l_y_1), (l_x_2, 800-l_y_2), 4)

        r_x_1, r_y_1 = self.drone.frame_shape.body.local_to_world((self.drone_radius, 0))
        r_x_2, r_y_2 = self.drone.frame_shape.body.local_to_world((self.drone_radius, self.froce_scale*vector_scale))
        pygame.draw.line(self.screen, (179,179,179), (r_x_1, 800-r_y_1), (r_x_2, 800-r_y_2), 4)

        r_x_2, r_y_2 = self.drone.frame_shape.body.local_to_world((self.drone_radius, self.right_force*vector_scale))
        pygame.draw.line(self.screen, (255,0,0), (r_x_1, 800-r_y_1), (r_x_2, 800-r_y_2), 4)

        pygame.draw.circle(self.screen, (255, 0, 0), (self.x_target, 800-self.y_target), 5)

        #Drawing drone's path
        if len(self.flight_path) > 2:
            pygame.draw.aalines(self.screen, (16, 19, 97), False, self.flight_path)

        if len(self.drop_path) > 2:
            pygame.draw.aalines(self.screen, (255, 0, 0), False, self.drop_path)

        pygame.display.flip()
        self.clock.tick(60)

    def initial_movement(self):
        if self.initial_throw is True:
            throw_angle = random.random() * 2*np.pi
            throw_force = random.uniform(0, 25000)
            throw = Vec2d(np.cos(throw_angle)*throw_force, np.sin(throw_angle)*throw_force)

            self.drone.frame_shape.body.apply_force_at_world_point(throw, self.drone.frame_shape.body.position)

            throw_rotation = random.uniform(-3000, 3000)
            self.drone.frame_shape.body.apply_force_at_local_point(Vec2d(0, throw_rotation), (-self.drone_radius, 0))
            self.drone.frame_shape.body.apply_force_at_local_point(Vec2d(0, -throw_rotation), (self.drone_radius, 0))

            self.space.step(1.0/60)
            if self.render_sim is True and self.render_path is True: self.add_postion_to_drop_path()

        else:
            throw_angle = None
            throw_force = None
            throw_rotation = None

        initial_stabilisation_delay = self.stabilisation_delay
        while self.stabilisation_delay != 0:
            self.space.step(1.0/60)
            if self.render_sim is True and self.render_path is True: self.add_postion_to_drop_path()
            if self.render_sim is True: self.render()
            self.stabilisation_delay -= 1

        self.stabilisation_delay = initial_stabilisation_delay

        return {'throw_angle': throw_angle, 'throw_force': throw_force, 'throw_rotation': throw_rotation}

    def add_postion_to_drop_path(self):
        x, y = self.drone.frame_shape.body.position
        self.drop_path.append((x, 800-y))

    def add_postion_to_flight_path(self):
        x, y = self.drone.frame_shape.body.position
        self.flight_path.append((x, 800-y))

    def add_drone_shade(self):
        x, y = self.drone.frame_shape.body.position
        self.path_drone_shade.append([x, y, self.drone.frame_shape.body.angle])
        self.shade_x = x
        self.shade_y = y

    def change_target_point(self, x, y):
        self.x_target = x
        self.y_target = y

Implementing Proximal Policy Optimization A2C with pytorch


In [ ]:
#Importing Neccessary Libraries for the model
import os
import numpy as np
import time
import glob
from datetime import datetime
import gym
#-----------pytorch libraries----------------#
import torch
import torch.nn as nn
from torch.distributions import MultivariateNormal
from scipy.stats import multivariate_normal
from torch.optim import Adam

In [ ]:
#Implementing the Proximal Policy Optimization
#The below class will store the local variables for clean policy

class store:
  def __init__(self):
    self.actions=[]
    self.states=[]
    self.log_probs=[]
    self.rewards=[]
    self.state_values=[]
    self.terminal=[]

  def clear(self):
    del self.actions[:]
    del self.states[:]
    del self.log_probs[:]
    del self.rewards[:]
    del self.state_values[:]
    del self.terminal[:]

#Implementing A2C
#since we are dealing with Drone environment with continuous space of action
#i.e [0,1]^4

class Actor_Critic(nn.Module):
  def __init__(self,action_dim,state_dim,action_dist):
    super(Actor_Critic,self).__init__()

    self.action_var=torch.full((action_dim,),action_dist*action_dist).to(device)
    self.action_dim = action_dim
    #Actor Network
    self.Actor=nn.Sequential(
        nn.Linear(state_dim,84),
        nn.ReLU(),
        nn.Linear(84,84),
        nn.ReLU(),
        nn.Linear(84,action_dim),
        nn.Tanh()
    )

    #Critic Network
    self.Critic=nn.Sequential(
        nn.Linear(state_dim,84),
        nn.ReLU(),
        nn.Linear(84,84),
        nn.Tanh(),
        nn.Linear(84,1)
    )

  def setup_action_dist(self,new_action_dist):
    #since we are dealing continuous space action we need to keep on changing the
    #action dist to make our model learn new things
    self.action_var=torch.full((self.action_dim,),new_action_dist*new_action_dist).to(device)

  def action_pred(self,obs):
    action_mean=self.Actor(obs)
    cov_mat=torch.diag(self.action_var).unsqueeze(dim=0)
    prob_distribution=MultivariateNormal(action_mean,cov_mat)
    action=prob_distribution.sample()
    log_probs=prob_distribution.log_prob(action)
    state_vals=self.Critic(obs)

    return action.detach(),log_probs.detach(),state_vals.detach()

  def evaluate(self,obs,action):
    values=self.Critic(obs)
    action_mean=self.Actor(obs)
    cov_mat=torch.diag_embed(self.action_var.expand_as(action_mean))
    prob_distribution=MultivariateNormal(action_mean,cov_mat)
    log_probs=prob_distribution.log_prob(action)

    return log_probs,values


#From here we will implement the clipped policy
class PPO:
  def __init__(self,action_dim,state_dim,epoch,lr_actor,lr_critic,clip,gamma,action_dist):
    self.states_dim=state_dim
    self.action_dim=action_dim
    self.action_dist=action_dist
    self.epoch=epoch
    self.clip=clip
    self.gamma=gamma
    self.lr_actor=lr_actor
    self.lr_critic=lr_critic
    self.local=store()
    self.policy=Actor_Critic(self.action_dim,self.states_dim,action_dist).to(device)
    self.optimizer=torch.optim.Adam([
        {'params':self.policy.Actor.parameters(),'lr':self.lr_actor},
        {'params':self.policy.Critic.parameters(),'lr':self.lr_critic}
    ])
    self.policy_old=Actor_Critic(self.action_dim,self.states_dim,action_dist).to(device)
    self.policy_old.load_state_dict(self.policy.state_dict())

  def set_action_dist_policy(self,new_action_dist):
    self.action_dist=new_action_dist
    self.policy.setup_action_dist(new_action_dist)
    self.policy_old.setup_action_dist(new_action_dist)

  def decay_action_dist(self,action_dist_decay_rate,min_action_dist):
    print("-----------setting-up-action-decay-rate-------------")
    self.action_dist=self.action_dist-action_dist_decay_rate
    if(self.action_dist<=min_action_dist):
      self.action_dist=min_action_dist
      print("setting the action dist to min_action_dist:",min_action_dist)

    else:
      print("setting the action dist to: ",self.action_dist)

    self.set_action_dist_policy(self.action_dist)

  def pick_action(self,obs):
    with torch.no_grad():
      obs=torch.FloatTensor(obs).to(device)
      action,logprob,state_val=self.policy_old.action_pred(obs)

    self.local.actions.append(action)
    self.local.log_probs.append(logprob)
    self.local.states.append(obs)
    self.local.state_values.append(state_val)

    return action.detach().cpu().numpy().flatten()

  def update(self):
    rewards=[]
    discounted_reward=0
    for reward, terminal in zip(reversed(self.local.rewards),reversed(self.local.terminal)):
        if terminal:
            discounted_reward=0
        discounted_reward=reward+(self.gamma*discounted_reward)
        rewards.insert(0,discounted_reward)

    rewards=torch.tensor(rewards,dtype=torch.float32).to(device)
    rewards=(rewards-rewards.mean())/(rewards.std()+1e-7)

    stack_states=torch.squeeze(torch.stack(self.local.states,dim=0)).detach().to(device)
    stack_actions=torch.squeeze(torch.stack(self.local.actions,dim=0)).detach().to(device)
    stack_log_probs=torch.squeeze(torch.stack(self.local.log_probs,dim=0)).detach().to(device)
    stack_state_values=torch.squeeze(torch.stack(self.local.state_values,dim=0)).detach().to(device)

    advantages=rewards.detach()-stack_state_values.detach()

    for i in range(self.epoch):
      logprobs,state_val=self.policy.evaluate(stack_states,stack_actions)
      state_val=torch.squeeze(state_val)
      ratio=torch.exp(logprobs-stack_log_probs)
      s1=ratio*advantages
      s2=torch.clamp(ratio,1-self.clip,1+self.clip)*advantages

      loss=-torch.min(s1,s2) +0.5*nn.MSELoss()(state_val,rewards)

      self.optimizer.zero_grad()
      loss.mean().backward()
      self.optimizer.step()

    self.policy_old.load_state_dict(self.policy.state_dict())
    self.local.clear()

  def save(self,path):
    torch.save(self.policy_old.state_dict(),path)

  def load(self,path):
    self.policy_old.load_state_dict(torch.load(path, map_location=lambda storage, loc: storage))
    self.policy.load_state_dict(torch.load(path, map_location=lambda storage, loc: storage))


In [ ]:
if __name__ == "__main__":
    from gym.envs.registration import register

    register(
        id='drone-2d-custom-v0',
        entry_point='__main__:Drone2dEnv',
        kwargs={'render_sim': False, 'render_path': True, 'render_shade': True,
                'shade_distance': 75, 'n_steps': 500, 'n_fall_steps': 10, 'change_target': False,
                'initial_throw': True}
    )

    # Create an instance of the Drone2dEnv class
    env = Drone2dEnv()

    # Perform any necessary initialization or setup here

    # Run the environment loop
    done = False
    observation = env.reset()
    while not done:
        action = env.action_space.sample()  # Replace with your desired action selection logic
        observation, reward, done, info = env.step(action)
          # Optional: Render the environment

In [ ]:
#setting up the process to cpu to cuda, please select GPU in runtime,
#if you are doing it in colab!
print("-----------setting-up-device-------------")
if(torch.cuda.is_available()):
  device=torch.device('cuda:0')
  torch.cuda.empty_cache() #clearing out the cache in the begining
  print("The process will be fast as Device Is Set to: "+ str(torch.cuda.get_device_name(device)))

else:
  device=torch.device('cpu')
  print("The process will be slow as Device is set to: CPU")
print("-----------------------------------------")
def train():
  print("-------initiating training of policy-----")
  env_name="Drone_RL_PPO_V1.2"
  print("-------Env has continuos action space----")
  max_epi_len=1500
  max_training_timestep=int(2e6)

  print_freq=max_epi_len*10
  log_freq=max_epi_len*2
  save_model_freq=int(5e5)

  action_dist=0.6
  action_dist_decay_rate=0.06
  min_action_dist=0.1
  action_std_decay_freq=int(2e5)

  update_timestep=max_epi_len*4
  epoch=60
  clip=0.2
  gamma=0.98
  lr_actor=0.0003
  lr_critic=0.001
  seed=0
  print("-----------------------------------------")
  print("ENV name: "+env_name)
  state_dim=env.observation_space.shape[0]
  action_dim=env.action_space.shape[0]
  log_dir="PPO_LOGS"
  if not os.path.exists(log_dir):
        os.makedirs(log_dir)

  log_dir = log_dir + '/' + env_name + '/'
  if not os.path.exists(log_dir):
        os.makedirs(log_dir)

  log_f_name=log_dir+env_name+"_log_01"+".csv"
  print("logging at: "+log_f_name)

  num_pre=0
  directory = "PPO_preTrained"
  if not os.path.exists(directory):
        os.makedirs(directory)

  directory = directory + '/' + env_name + '/'
  if not os.path.exists(directory):
        os.makedirs(directory)

  path=directory+"PPO_{}_{}.pth".format(env_name,num_pre)
  print("model saving at: " +path)

  torch.manual_seed(seed)
  np.random.seed(seed)
  env.seed(seed)
  print("-----------------------------------------")

  agent=PPO(action_dim,state_dim,epoch,lr_actor,lr_critic,clip,gamma,action_dist)
  start=datetime.now()
  print("started training time at: ",start)
  logger=open(log_f_name,"w+")
  logger.write('Episode,Timestep,Reward\n')

  log_running_reward=0
  log_running_episodes=0
  print_running_reward=0
  print_running_episodes=0
  time_step=0
  i_episode=0

  while time_step<=max_training_timestep:
    state=env.reset()
    current_episode_reward=0
    for i in range(1,max_epi_len+1):
      action=agent.pick_action(state)
      state,reward,done,_=env.step(action)
      agent.local.rewards.append(reward)
      agent.local.terminal.append(done)
      time_step+=1
      current_episode_reward+=reward

      if time_step % update_timestep==0:
        agent.update()

      if time_step % action_std_decay_freq==0:
        agent.decay_action_dist(action_dist_decay_rate,min_action_dist)

      if time_step % log_freq==0:
        log_avg_reward=log_running_reward / log_running_episodes
        log_avg_reward=round(log_avg_reward,4)

        logger.write('{},{},{}\n'.format(i_episode, time_step, log_avg_reward))
        logger.flush()

        log_running_reward=0
        log_running_episodes=0

      if time_step % print_freq==0:
        print_avg_reward=print_running_reward / print_running_episodes
        print_avg_reward=round(print_avg_reward, 2)

        print("Episode : {} \t\t Timestep : {} \t\t Average Reward : {}".format(i_episode, time_step, print_avg_reward))

        print_running_reward=0
        print_running_episodes=0

      if time_step % save_model_freq==0:
        print("-----------------------------------------")
        print("saving model at: "+path)
        agent.save(path)
        print("model saved")
        print("Time taken: ",datetime.now().replace(microsecond=0) - start)
        print("-----------------------------------------")

      if done:
        break
    print_running_reward+=current_episode_reward
    print_running_episodes+=1
    log_running_reward += current_episode_reward
    log_running_episodes += 1
    i_episode += 1

  logger.close()
  env.close()
  print("-----------------------------------------")
  end_time = datetime.now().replace(microsecond=0)
  print("Started training at (GMT) : ", start)
  print("Finished training at (GMT) : ", end_time)
  print("Total training time  : ", end_time - start)
  print("-----------------------------------------")

if __name__ =='__main__':
  train()